**Creacion de DataFrames auxiliares**

En este notebook vamos a crear los data frames auxiliares que tendremos que usar para las funciones que vamos a desarrollar

**Importaciones**

In [5]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

**Cargamos los CSV necesarios para extraer la informacion necesaria**

In [6]:
df_reviews = pd.read_csv("../Archivos Json/user_reviews_limpio.csv", encoding="utf-8")
df_items = pd.read_csv("../Archivos Json/user_items_limpio.csv", encoding="utf-8")
df_games = pd.read_csv("../Archivos Json/steam_games_limpio.csv", encoding="utf-8")

In [7]:
df_items.columns

Index(['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever'], dtype='object')

In [8]:
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'year_posted', 'sentiment_analysis'], dtype='object')

In [9]:
df_games.columns

Index(['app_name', 'id', 'developer', 'price_number', 'release_year',
       'main_genre'],
      dtype='object')

In [11]:
df_items["playtime_hour"] = df_items["playtime_forever"]/60
df_items

,user_id,items_count,item_id,item_name,playtime_forever,playtime_hour
0,76561197970982479,277,20,Team Fortress Classic,0.0,0.000000
1,76561197970982479,277,30,Day of Defeat,7.0,0.116667
2,76561197970982479,277,40,Deathmatch Classic,0.0,0.000000
3,76561197970982479,277,60,Ricochet,0.0,0.000000
4,76561197970982479,277,3830,Psychonauts,333.0,5.550000
...,...,...,...,...,...,...
3002796,76561198329548331,7,304930,Unturned,677.0,11.283333
3002797,76561198329548331,7,346330,BrainBread 2,0.0,0.000000
3002798,76561198329548331,7,373330,All Is Dust,0.0,0.000000
3002799,76561198329548331,7,521570,You Have 10 Seconds 2,4.0,0.066667


**Funcion PlayTimeGenre**

En este caso nos piden una funcion que devuelva el año con mas horas jugadas para un genero especificado, por lo que seran necesarias las columnas main_genre, id y Release_year de df_output y la columna playtime_forever e item_id de df_items

In [12]:
df_output2 = df_games[["main_genre", "release_year","id"]]
df_items2 = df_items[["playtime_hour", "item_id"]]
df_PlayTimeGenre_merge = pd.merge(df_items2, df_output2, left_on='item_id', right_on="id")
df_PlayTimeGenre_hour = df_PlayTimeGenre_merge.groupby(["main_genre", "release_year"])["playtime_hour"].sum().reset_index()
df_PlayTimeGenre_hour

,main_genre,release_year,playtime_hour
0,Action,1984,6.400000
1,Action,1990,313.116667
2,Action,1991,19.116667
3,Action,1992,12.683333
4,Action,1993,33.700000
...,...,...,...
230,Utilities,2016,1305.183333
231,Video Production,2015,0.000000
232,Web Publishing,2005,1564.116667
233,Web Publishing,2015,3.100000


Guardamos el nuevo dataframe como un CSV

In [13]:
archivo_limpio = "../Archivos Json/df_PlayTimeGenre_hour_final1.csv"
df_PlayTimeGenre_hour.to_csv(archivo_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio}") 

Se guardó el archivo ../Archivos Json/df_PlayTimeGenre_hour_final1.csv


Guardamos el nuevo dataframe como archivo PARQUET

In [14]:
archivo_limpio2 = "../Datasets/df_PlayTimeGenre_hour_final1.parquet"
df_PlayTimeGenre_hour.to_parquet(archivo_limpio2, index=False)
print(f"Se guardó el archivo {archivo_limpio2}")

Se guardó el archivo ../Datasets/df_PlayTimeGenre_hour_final1.parquet


**Funcion UserForGenre**

En este caso nos piden una funcion que devuelva el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año, por lo que seran necesarias las columnas "item_id","user_id" y "year_posted" de df_reviews, como tambien las columnas "user_id", "item_id", "playtime_hour" de df_items y "id" e "main_genre" de df_games

Filtramos las columnas de df_reviews y de df_items necesarias

In [15]:
data_reviews_subset = df_reviews[["item_id","user_id"]]

In [16]:
data_items_subset = df_items[["user_id","item_id","playtime_hour"]]

Unimos las columnas que fueron previamente filtradas en un solo dataframe

In [17]:
merged_data = data_items_subset.merge(data_reviews_subset, on=["user_id", "item_id"], how="inner")

Al dataframe ya unido previamente le unimos las columnas necesarias de df_output

In [18]:
final_merged_data = merged_data.merge(df_games[["id","main_genre","release_year"]], left_on="item_id", right_on="id", how="inner")

Agrupamos apartir de "user_id", "main_genre" y "year_posted" y despues sumamos las horas de juegos acumuladas por estos grupos

In [19]:
df_UsersForGenre = final_merged_data.groupby(["user_id","main_genre","release_year"])["playtime_hour"].sum().reset_index()

In [20]:
df_UsersForGenre

,user_id,main_genre,release_year,playtime_hour
0,--000--,Action,2009,49.150000
1,--ace--,Action,2011,21.150000
2,--ionex--,Action,2011,14.033333
3,-Azsael-,Indie,2015,198.150000
4,-Beave-,Action,2009,47.733333
...,...,...,...,...
21504,zyr0n1c,Action,2008,1.116667
21505,zyr0n1c,Action,2009,29.633333
21506,zyr0n1c,Action,2011,10.216667
21507,zyr0n1c,Action,2013,153.200000


In [21]:
data_reviews_subset2 = df_reviews[["item_id","user_id","year_posted"]]
data_items_subset2 = df_items[["user_id","item_id","playtime_hour"]]
merged_data2 = data_items_subset2.merge(data_reviews_subset2, on=["user_id", "item_id"], how="inner")
final_merged_data2 = merged_data2.merge(df_games[["id","main_genre"]], left_on="item_id", right_on="id", how="inner")
df_UsersForGenre2 = final_merged_data2.groupby(["user_id","main_genre","year_posted"])["playtime_hour"].sum().reset_index()
df_UsersForGenre2

,user_id,main_genre,year_posted,playtime_hour
0,--000--,Action,2014,49.150000
1,--ace--,Action,2014,21.150000
2,--ionex--,Action,2015,14.033333
3,-Azsael-,Indie,2013,198.150000
4,-Beave-,Action,2014,47.733333
...,...,...,...,...
19124,zvanik,Action,2015,382.233333
19125,zynxgameth,Action,2013,64.100000
19126,zyr0n1c,Action,2013,40.966667
19127,zyr0n1c,Action,2014,10.716667


In [22]:
archivo_limpio001 = "../Archivos Json/df_UsersForGenre3_final.csv"
df_UsersForGenre2.to_csv(archivo_limpio001, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio001}") 

Se guardó el archivo ../Archivos Json/df_UsersForGenre3_final.csv


Guardamos el nuevo dataframe como un CSV

In [23]:
archivo_limpio00 = "../Archivos Json/df_UsersForGenre2_final.csv"
df_UsersForGenre.to_csv(archivo_limpio00, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio00}") 

Se guardó el archivo ../Archivos Json/df_UsersForGenre2_final.csv


Guardamos el nuevo dataframe como archivo PARQUET

In [24]:
archivo_limpio000 = "../Datasets/df_UsersForGenre2_final.parquet"
df_UsersForGenre.to_parquet(archivo_limpio000, index=False)
print(f"Se guardó el archivo {archivo_limpio000}")

Se guardó el archivo ../Datasets/df_UsersForGenre2_final.parquet


**Funcion UsersRecommend**

En este caso nos piden una funcion que devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado, por lo que seran necesarias la columna id y app_name de df_games y las columnas item_id, recommend y year_posted de df_reviews

In [25]:
df_output3_ = df_games[["id", "app_name"]].drop_duplicates()
df_reviews2_ = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged_ = pd.merge(df_reviews2_, df_output3_, left_on="item_id", right_on="id")
positive_reviews_ = df_reviews_merged_[(df_reviews_merged_["recommend"]== True) & (df_reviews_merged_["sentiment_analysis"] >=1)]
grouped_reviews_ = positive_reviews_.groupby(["year_posted", "item_id", "app_name"]).size().reset_index(name="recommendation_count")
df_UsersRecommend_ = (
    grouped_reviews_.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersRecommend_

,year_posted,item_id,app_name,recommendation_count
0,2010,440,Team Fortress 2,10
1,2010,1250,Killing Floor,6
2,2010,630,Alien Swarm,4
3,2011,440,Team Fortress 2,72
4,2011,620,Portal 2,24
5,2011,105600,Terraria,20
6,2012,440,Team Fortress 2,234
7,2012,4000,Garry's Mod,36
8,2012,105600,Terraria,36
9,2013,440,Team Fortress 2,938


Guardamos el nuevo dataframe como un CSV

In [26]:
archivo_limpio5 = "../Archivos Json/df_UsersRecommend2_final.csv"
df_UsersRecommend_.to_csv(archivo_limpio5, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio5}") 

Se guardó el archivo ../Archivos Json/df_UsersRecommend2_final.csv


Guardamos el nuevo dataframe como archivo PARQUET

In [27]:
archivo_limpio6 = "../Datasets/df_UsersRecommend2_final.parquet"
df_UsersRecommend_.to_parquet(archivo_limpio6, index=False)
print(f"Se guardó el archivo {archivo_limpio6}")

Se guardó el archivo ../Datasets/df_UsersRecommend2_final.parquet


**Funcion UsersWorstDeveloper**

En este caso nos piden una funcion que devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos). Se utilizaran las columnas "id" y "developer" del df_games, tambien las columnas "item_id", "recommend", "year_posted" y "sentiment_analysis"del df_reviews

In [28]:
df_output4 = df_games[["id", "developer"]].drop_duplicates()
df_reviews4 = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged4 = pd.merge(df_reviews4, df_output4, left_on="item_id", right_on="id")
negative_reviews4 = df_reviews_merged4[(df_reviews_merged4["recommend"]== False) & (df_reviews_merged4["sentiment_analysis"] == 0)]
grouped_reviews4 = negative_reviews4.groupby(["year_posted", "item_id", "developer"]).size().reset_index(name="recommendation_count")
df_UsersWorstDeveloper = (
    grouped_reviews4.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersWorstDeveloper

,year_posted,item_id,developer,recommendation_count
0,2011,440,Valve,1
1,2011,18700,Broken Rules,1
2,2011,33460,Ubisoft Montpellier,1
3,2012,440,Valve,1
4,2012,42920,NeoCoreGames,1
5,2012,55110,Volition,1
6,2013,275850,Hello Games,52
7,2013,730,Valve,32
8,2013,440,Valve,25
9,2014,221100,Bohemia Interactive,19


In [29]:
df_UsersWorstDeveloper_final = df_UsersWorstDeveloper.groupby(["year_posted","developer"], as_index=False)["recommendation_count"].sum()
df_UsersWorstDeveloper_final

,year_posted,developer,recommendation_count
0,2011,Broken Rules,1
1,2011,Ubisoft Montpellier,1
2,2011,Valve,1
3,2012,NeoCoreGames,1
4,2012,Valve,1
5,2012,Volition,1
6,2013,Hello Games,52
7,2013,Valve,57
8,2014,Bohemia Interactive,19
9,2014,"Trek Industries, Inc",14


Guardamos el nuevo dataframe como un CSV

In [31]:
archivo_limpio7 = "../Archivos Json/df_UsersWorstDeveloper_final1.csv"
df_UsersWorstDeveloper_final.to_csv(archivo_limpio7, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio7}") 

Se guardó el archivo ../Archivos Json/df_UsersWorstDeveloper_final1.csv


Guardamos el nuevo dataframe como archivo PARQUET

In [32]:
archivo_limpio8 = "../Datasets/df_UserWorstDeveloper_final1.parquet"
df_UsersWorstDeveloper.to_parquet(archivo_limpio8, index=False)
print(f"Se guardó el archivo {archivo_limpio8}")

Se guardó el archivo ../Datasets/df_UserWorstDeveloper_final1.parquet


**Funcion Sentiment_analysis**

En este caso nos piden una funcion que devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. Se utilizan todas las columnas de df_reviews y las columnas "id","app_name" y "developer" de df_output para finalmente quedarnos con las columnas developer y sentiment_analysis y sobre estas columnas hacer un conteo del analisis de sentimiento

In [33]:
df_reviews0 = df_reviews[["user_id", "item_id","recommend","year_posted","sentiment_analysis"]]
df_reviews1 = pd.merge(df_reviews0, df_games[["id","app_name","developer"]], left_on="item_id", right_on="id")
df_reviews02 = df_reviews1[["user_id","app_name","developer","recommend","year_posted","sentiment_analysis"]]
df_Sentiment_analysis = df_reviews02.groupby(["developer","sentiment_analysis"]).size().reset_index(name="sentiment_analysis_count")
df_Sentiment_analysis

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",1,1
4,10tons Ltd,1,1
...,...,...,...
3977,"インレ,Inre",1,4
3978,橘子班,0,1
3979,橘子班,1,2
3980,橘子班,2,1


Guardamos el nuevo dataframe como un CSV

In [35]:
archivo_limpio10 = "../Archivos Json/df_Sentiment_analysis_final.csv"
df_Sentiment_analysis.to_csv(archivo_limpio10, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio10}") 

Se guardó el archivo ../Archivos Json/df_Sentiment_analysis_final.csv


Guardamos el nuevo dataframe como archivo PARQUET

In [36]:
archivo_limpio11 = "../Datasets/df_Sentiment_analysis_final.parquet"
df_Sentiment_analysis.to_parquet(archivo_limpio11, index=False)
print(f"Se guardó el archivo {archivo_limpio11}")

Se guardó el archivo ../Datasets/df_Sentiment_analysis_final.parquet
